In [83]:
from sklearn.pipeline import Pipeline, make_pipeline
from dask_ml.preprocessing import Categorizer, DummyEncoder, StandardScaler
from dask_ml.model_selection import train_test_split
import pandas as pd
import dask.dataframe as dd
#from dask_ml.linear_model import LinearRegression
from sklearn.linear_model import LinearRegression, Lasso, Ridge

In [53]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

/anaconda3/lib/python3.7/site-packages/distributed/bokeh/core.py:57: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


Client Scheduler: inproc://192.168.1.12/2036/8 Dashboard: http://localhost:51693/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


In [54]:
#day = dd.read_csv('Bike-Sharing-Dataset/day.csv')
#hour = dd.read_csv('Bike-Sharing-Dataset/hour.csv')
day = dd.read_csv('https://gist.githubusercontent.com/louisdubaere/44d9cae2acbdb4c45144da05e62e677e/raw/dfbc17b394dbe3f966111fd4e2636b0184413b0c/day.csv')
hour = dd.read_csv('https://gist.githubusercontent.com/louisdubaere/7b76bb68cbde26bfd9fcfefe0d498b75/raw/4fe5ee53f3f329b902ded808a5367066c86f0cb9/hour.csv')

In [55]:
hour.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


In [56]:
hour.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 17 entries, instant to cnt
dtypes: object(1), float64(4), int64(12)

In [57]:
hour.dtypes

instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

In [59]:
X = hour.drop(['casual', 'registered', 'cnt','dteday','instant','temp'], axis = 1)
y = hour['cnt']

In [66]:
length = client.submit(len, X).result()
train_n = round(0.8 * length)
X_train = X[0:train_n]
X_test = X[train_n+1:length]
y_train = y.loc[0:train_n]
y_test = y.loc[train_n+1:length]

In [67]:
categories = ['season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit']
scale = ['atemp', 'hum', 'windspeed']

In [73]:
pipe = make_pipeline(Categorizer(columns = categories), DummyEncoder(), StandardScaler(scale), LinearRegression())

In [74]:
fitted_pipe = client.submit(pipe.fit, X_train, y_train)
score = client.submit(fitted_pipe.result().score, X_test, y_test)
score.result()

0.6336091686562537

In [70]:
pipe.fit(X_train,y_train)
pipe.score(X_test, y_test)

0.6336091686562537

In [94]:
from dask_ml.model_selection import GridSearchCV

lasso = Lasso()
ridge = Ridge()

parameters = {'alpha': [1e-15, 1e-10, 1e-8, 1e-4, 1e-3, 1e-2, 1, 2, 5]}

ridge_regression = GridSearchCV(ridge, parameters, cv=5)
lasso_regression = GridSearchCV(lasso, parameters, cv=5)


In [95]:
fitted_ridge = client.submit(ridge_regression.fit, X_train, y_train)
score = client.submit(fitted_ridge.result().score, X_test, y_test)
score.result()

0.30835977015510263

In [96]:
fitted_lasso = client.submit(lasso_regression.fit, X_train, y_train)
score = client.submit(fitted_lasso.result().score, X_test, y_test)
score.result()

/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of

0.29419230442466704